<a href="https://colab.research.google.com/github/leenago/NLP_edu/blob/master/200904_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_Naive_Bayes_%EA%B8%8D%EC%A0%95%EB%B6%80%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Practice
- 긍정(1), 부정(0)

### 1) Google drive에 mount 시켜서 파일 불러오기

In [9]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)   

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
%ls

 실습코드/                          단어의표현_part1.pdf
 1_단어의표현_실습3_TF_IDF_1.gdoc   단어의표현_part2.pdf
'1_자연어처리 기초.pdf'             감성분석_답.pdf
 2_텍스트마이닝.pdf                 감성분석_나이브베이지안예제풀이.pdf
 naivebayes_example.csv             감성분석.pdf
 단어의표현_part1.gdoc


In [11]:
%cd /content/drive/My Drive/bigdata/아경/2020_09_청취 아경_산업융합빅데이터분석가양성/자연어처리

/content/drive/.shortcut-targets-by-id/16DQB6HAwJs4icviMPd6AYLFaIjgdJaQZ/2020_09_청취 아경_산업융합빅데이터분석가양성/자연어처리


In [12]:
%ls

 실습코드/                          단어의표현_part1.pdf
 1_단어의표현_실습3_TF_IDF_1.gdoc   단어의표현_part2.pdf
'1_자연어처리 기초.pdf'             감성분석_답.pdf
 2_텍스트마이닝.pdf                 감성분석_나이브베이지안예제풀이.pdf
 naivebayes_example.csv             감성분석.pdf
 단어의표현_part1.gdoc


### 2) 파일 불러오기

In [17]:
import numpy as np
import pandas as pd
input_file = pd.read_csv('naivebayes_example.csv')
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [20]:
training_set = np.array(input_file)
training_set

array([['i love you', 1],
       ['love happy weekend', 1],
       ['bore work job', 0],
       ['i hate you', 0],
       ['bore weekend', 0],
       ['happy together', 1]], dtype=object)

### 3) 토큰 빈도수 및 문서별 토큰수 계산
- 확률 계산을 위한 준비

In [23]:
#토큰별로 문서 내 빈도수 카운팅
from collections import defaultdict

wordfreq = defaultdict(lambda: [0,0])
#lambda: 새로운 단어가 추가 될 때 default를 [0:0]으로 하겠다. (긍정, 부정)

for doc, point in training_set:
  words = doc.split() #단어 토큰화

  for word in words:
    if point == 1:
      wordfreq[word][0] += 1 #긍정인 경우 0번째 인덱스에 1을 더해라
    else:
      wordfreq[word][1] += 1 #부정인 경우 1번째 인덱스에 1을 더해라

wordfreq

defaultdict(<function __main__.<lambda>>,
            {'bore': [0, 2],
             'happy': [2, 0],
             'hate': [0, 1],
             'i': [1, 1],
             'job': [0, 1],
             'love': [2, 0],
             'together': [1, 0],
             'weekend': [1, 1],
             'work': [0, 1],
             'you': [1, 1]})

In [24]:
#긍정/부정 빈도수 계산
긍정전체토큰수 = []
부정전체토큰수 = []

for key, (cnt1, cnt0) in wordfreq.items():
  #key: 토큰화 된 단어들 / cnt1, cnt0 = 긍정/부정인 경우에 더해졌던 인덱스들
  긍정전체토큰수.append(int(cnt1))
  부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)
print('\n')

전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정, 전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]


8 8


### 4) 토큰별 조건부 확률 계산

In [25]:
wordprobs = defaultdict(lambda:[0,0])

for key, (cnt1,cnt0) in wordfreq.items():
  wordprobs[key][0] = (cnt1+1) / (전체갯수_긍정 + len(wordfreq))
  wordprobs[key][1] = (cnt0+1) / (전체갯수_부정 + len(wordfreq))
  #len(wordfreq): 전체 unique한 토큰 개수
  #Laplace Smoothing을 사용
  
wordprobs

defaultdict(<function __main__.<lambda>>,
            {'bore': [0.05555555555555555, 0.16666666666666666],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'i': [0.1111111111111111, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'together': [0.1111111111111111, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'work': [0.05555555555555555, 0.1111111111111111],
             'you': [0.1111111111111111, 0.1111111111111111]})

### 5) Classify: 신규텍스트가 주어졌을 때 확률계산

In [26]:
import math
doc = 'happy weekend'
tokens = doc.split()
tokens

['happy', 'weekend']

In [28]:
#초기값을 모두 0으로 처리
log_prob1 = log_prob0 = 0.0

#모든 단어에 대해 반복
for word, (prob1, prob0) in wordprobs.items():
  if word in tokens:
    log_prob1 += math.log(prob1)
    log_prob0 += math.log(prob0)

log_prob1 += math.log(전체갯수_긍정/(전체갯수_긍정+전체갯수_부정))
log_prob0 += math.log(전체갯수_부정/(전체갯수_긍정+전체갯수_부정))

prob1 = math.exp(log_prob1)
print('prob1: ', prob0)

prob0 = math.exp(log_prob0)
print('prob0: ', prob0)
print('\n')

print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률')
print('긍정확률: {}%'.format(prob1/(prob1+prob0)*100))
print('부정확률: {}%'.format(prob0/(prob1+prob0)*100))

prob1:  0.05555555555555555
prob0:  0.0030864197530864183


happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률
긍정확률: 75.00000000000001%
부정확률: 24.999999999999996%


## Using sklearn

In [13]:
import pandas as pd
input_file = pd.read_csv('naivebayes_example.csv')

In [14]:
X_train = list(input_file['mail'])
Y_train = list(input_file['label'])
print(X_train, Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together'] [1, 1, 0, 0, 0, 1]


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()

#fit and transform
X_train_counts = count_vect.fit_transform(X_train)

#MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [16]:
happy_weekend = ['happy weekend'] 
#ValueError: Iterable over raw text documents expected, string object received.
#그냥 'happy weekend'를 집어넣으면 이런 오류가 떠서, list 안에 한번 더 넣어주었다.

#예측
print(clf.predict(count_vect.transform(happy_weekend)))
#확률
print(clf.predict_proba(count_vect.transform(happy_weekend)))

[1]
[[0.25 0.75]]
